# Connexion à la base de données

In [4]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('Files/devia-f6704-firebase-adminsdk-ay7ci-26941e7999.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

---
---

# Ajout des données

### Importer les données

In [5]:
import os, json

with open ("Objet_JSON_films.json", "r") as file:
    datas = json.load(file)

---

### Upload des données sur la BDD Firebase

In [6]:
for film in datas["films"]:

    db.collection('films').document(film["title"].replace(" ", "_")).set({key: film[key] for key in ["title", "year", "genre", "country"]})


    for director in film["director"]:

        db.collection('directors').document(f"{director['last_name']}_{director['first_name']}").set(director)

    for actor in film["actors"]:
        db.collection('actors').document(f"{actor['last_name']}_{actor['first_name']}").set({key : actor[key] for key in ["last_name", "first_name", "birth_date"]})
        db.collection('films').document(film["title"].replace(" ", "_")).collection('actors').document(f"{actor['last_name']}_{actor['first_name']}").set({"role" : actor["role"], "info" : db.collection('actors').document(f"{actor['last_name']}_{actor['first_name']}")})




for film in datas["films"]:

    for director in film["director"]:
        doc = db.collection('directors').document(f"{director['last_name']}_{director['first_name']}").get().to_dict()
        if "films" in list(doc):
            db.collection('directors').document(f"{director['last_name']}_{director['first_name']}").update({"films" : firestore.ArrayUnion([db.collection('films').document(film["title"].replace(" ", "_"))])})
        else :
            db.collection('directors').document(f"{director['last_name']}_{director['first_name']}").update({"films" : [db.collection('films').document(film["title"].replace(" ", "_"))]})

        doc = db.collection('films').document(film["title"].replace(" ", "_")).get().to_dict()
        if "directors" in list(doc.keys()):
            db.collection('films').document(film["title"].replace(" ", "_")).update({"directors" : firestore.ArrayUnion([db.collection('directors').document(f"{director['last_name']}_{director['first_name']}")])})
        else :
            db.collection('films').document(film["title"].replace(" ", "_")).update({"directors" : [db.collection('directors').document(f"{director['last_name']}_{director['first_name']}")]})


    for actor in film["actors"]:
        doc = db.collection('actors').document(f"{actor['last_name']}_{actor['first_name']}").get().to_dict()
        if "films" in list(doc):
            db.collection('actors').document(f"{actor['last_name']}_{actor['first_name']}").update({"films" : firestore.ArrayUnion([db.collection('films').document(film["title"].replace(" ", "_"))])})
        else :
            db.collection('actors').document(f"{actor['last_name']}_{actor['first_name']}").update({"films" : [db.collection('films').document(film["title"].replace(" ", "_"))]})

---
---

# Résultat

---

### Collection : films

![alternative text](Files/BDD_Films_01.jpg)
![alternative text](Files/BDD_Films_02.jpg)

---

### Collection : directors

![alternative text](Files/BDD_Films_03.jpg)

---

### Collection : actors

![alternative text](Files/BDD_Films_04.jpg)